In [560]:
import numpy as np
import pandas  as pd


from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


In [561]:

data = pd.read_csv("AB_NYC_2019.csv")

sample_data = data.sample(frac=0.1, random_state=42)

remaining_data = data.drop(sample_data.index)

sample_data_no_price = sample_data.drop(columns=['price'])
sample_data_no_price.to_csv("sample_no_price.csv", index=False)

remaining_data.to_csv("test_data.csv",index=False)

In [562]:
data.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129
6,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,2017-10-05,0.40,1,0
7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220
8,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,2017-07-21,0.99,1,0
9,5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-06-09,1.33,4,188


In [563]:
data=data.drop(['id', 'name', 'host_id', 'host_name', 'last_review', 'latitude', 'longitude'],axis=1)



In [ ]:



city_encoder = LabelEncoder()
data['city_code'] = city_encoder.fit_transform(data['neighbourhood_group']) * 10


data['location_code'] = -1  


for group_code in data['city_code'].unique():
    mask = data['city_code'] == group_code
    hood_encoder = LabelEncoder()
    hood_codes = hood_encoder.fit_transform(data.loc[mask, 'neighbourhood'])
    
    
    data.loc[mask, 'location_code'] = group_code + hood_codes


print(data[['neighbourhood_group', 'neighbourhood', 'location_code']].head())


  neighbourhood_group neighbourhood  location_code
0            Brooklyn    Kensington             41
1           Manhattan       Midtown             37
2           Manhattan        Harlem             30
3            Brooklyn  Clinton Hill             23
4           Manhattan   East Harlem             24


In [565]:
print(data.columns)

Index(['neighbourhood_group', 'neighbourhood', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'city_code',
       'location_code'],
      dtype='object')


In [566]:
print(data['location_code'])

0        41
1        37
2        30
3        23
4        24
         ..
48890    12
48891    20
48892    30
48893    31
48894    31
Name: location_code, Length: 48895, dtype: int64


In [567]:
data=data.drop(['neighbourhood_group', 'neighbourhood','city_code'],axis=1)




In [568]:
room_encoder = LabelEncoder()
data['room_type'] = room_encoder.fit_transform(data['room_type'])


In [569]:
data.head(5)

,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,location_code
0,1,149,1,9,0.21,6,365,41
1,0,225,1,45,0.38,2,355,37
2,1,150,3,0,NaN,1,365,30
3,0,89,1,270,4.64,1,194,23
4,0,80,10,9,0.10,1,0,24


In [570]:
print(data.columns)

Index(['room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'location_code'],
      dtype='object')


In [571]:
features = ['room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'location_code']

X = data[features]
y = data['price']

In [572]:
pip install seaborn joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [573]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.activations import swish


df = pd.read_csv("AB_NYC_2019.csv")


df = df[(df['price'] >= 10) & (df['price'] <= 500)]
df = df[(df['minimum_nights'] > 0) & (df['minimum_nights'] <= 30)]
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
df['days_since_review'] = (pd.to_datetime('2019-12-31') - df['last_review']).dt.days.fillna(365)
df['price_per_night'] = df['price'] / df['minimum_nights']
df['log_reviews'] = np.log1p(df['number_of_reviews'])


y = df['price']


features = ['room_type', 'neighbourhood_group', 'minimum_nights', 'log_reviews',
            'reviews_per_month', 'calculated_host_listings_count',
            'availability_365', 'days_since_review', 'price_per_night']
X = df[features]

ct = ColumnTransformer([
    ('num', StandardScaler(), ['minimum_nights', 'log_reviews', 'reviews_per_month',
                               'calculated_host_listings_count', 'availability_365',
                               'days_since_review', 'price_per_night']),
    ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['room_type', 'neighbourhood_group'])
])

X_transformed = ct.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)


model = Sequential()
model.add(Dense(64, activation='swish', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='swish'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])


stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.1, epochs=100, batch_size=32, callbacks=[stop], verbose=1)


pred = model.predict(X_test).flatten()
mae = mean_absolute_error(y_test, pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))

print(f"MAE (Ortalama Mutlak Hata): ${mae:.2f}")
print(f"RMSE (Karekök Ortalama Kare Hata): ${rmse:.2f}")


results = pd.DataFrame({'Gerçek Fiyat': y_test.values, 'Tahmin': pred})
print(results.head(10))


Epoch 1/100


c:\Users\Emre Gundogdu\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1061/1061 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 11900.5898 - mae: 77.2057 - val_loss: 2971.1792 - val_mae: 36.5327
Epoch 2/100
1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3015.6770 - mae: 37.2923 - val_loss: 2705.5361 - val_mae: 34.9940
Epoch 3/100
1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2619.2373 - mae: 34.4573 - val_loss: 2151.1450 - val_mae: 29.8120
Epoch 4/100
1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2282.1790 - mae: 31.3029 - val_loss: 1654.9543 - val_mae: 25.8425
Epoch 5/100
1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1856.0712 - mae: 27.8636 - val_loss: 1284.6907 - val_mae: 21.6913
Epoch 6/100
1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1589.3036 - mae: 25.0936 - val_loss: 1059.7943 - val_mae: 18.8118
Epoch 7/100
1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1336.4277 - mae: 23.1150 - val_loss: 981.4065 - val_mae: 16.1597
Epoch 8/100
1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1205.1693 - mae: 21.7769 - val_loss: 857.9446 